In [40]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Subset
from torch.nn import functional as F
import torchvision
from torchvision import transforms
import utils

### L2正則をスクラッチ実装

In [41]:
# フロベニアスノルム
X = torch.tensor([[1, 2, 3], [4, 5, 6]], dtype=torch.float32)
torch.sqrt(torch.sum(X**2))

tensor(9.5394)

In [42]:
X = torch.tensor([[1, 2, 3], [4, 5, 6]], dtype=torch.float32)
torch.linalg.norm(X)

tensor(9.5394)

In [43]:
def get_convmodel():
    return nn.Sequential(
    # 3x28x28 -> 4x14x14
    nn.Conv2d(3, 4, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    # 4x14x14 -> 8x7x7
    nn.Conv2d(4, 8, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    # 8x7x7 -> 16x3x3
    nn.Conv2d(8, 16, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    # 16x4x4 -> 32x1x1
    nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    #nn.AdaptiveAvgPool2d(1),   # <- GAP
    # 32
    nn.Flatten(),
    # 10
    nn.Linear(32, 10)
)

In [44]:
# L2ノルムを計算
conv_model = get_convmodel()
l2_reg = torch.tensor(0.)
for name, param in conv_model.named_parameters():
    if 'weight' in name:
        l2_reg += torch.linalg.norm(param)**2
# 正則化項を加えた損失
#loss += (weight_decay / (2*m)) * l2_reg

### PytorchでのL2正則

In [45]:
optim.SGD(conv_model.parameters(), lr=0.03, weight_decay=0.01)

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.03
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0.01
)

### L2正則有無でモデルを比較

In [46]:
conv_model = get_convmodel()
conv_model_l2 = get_convmodel()
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.40222, 0.4465], std=[0.2023, 0.1994, 0.2010])
])
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms)
val_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)

opt = optim.SGD(conv_model.parameters(), lr=0.03)
opt_l2 = optim.SGD(conv_model_l2.parameters(), lr=0.03, weight_decay=0.01)

Files already downloaded and verified
Files already downloaded and verified


In [47]:
num_epochs = 10
train_losses, val_losees, val_accuracies = utils.learn(conv_model, train_loader, val_loader, opt, F.cross_entropy, num_epoch=num_epochs)
train_losses_l2, val_losees_l2, val_accuracies_2 = utils.learn(conv_model_l2, train_loader, val_loader, opt_l2, F.cross_entropy, num_epoch=num_epochs)

TypeError: 'module' object is not callable

## DROP OUT

### スクラッチ実装

In [48]:
def dropout(X, drop_p):
    keep_p = 1 - drop_p
    mask = torch.rand(X.shape) < keep_p
    return X * mask / keep_p

### PyTorchのDropoutの実装

In [50]:
model = nn.Sequential(
    nn.Linear(64, 20),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(20, 10)
)